Want to do a quick recalc of yesterday's model using the sklearn API Again..

In [3]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib.pyplot as plt
# from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, LabelBinarizer
from sklearn.datasets import load_svmlight_file
from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score
import fresh.utils as fu

from importlib import reload
from collections import Counter
from tqdm.notebook import tqdm
import fresh.preproc.v1 as pv1
import fresh.preproc.v2 as pv2

In [8]:
localdir = '/home/ec2-user/SageMaker/learn-citibike'  # sagemaker
datadir = f'{localdir}/artifacts/2020-07-03T171842Z'

train_loc = f'{datadir}/train.libsvm'

# Convert the dtrain to numpy  ( nice advice from https://stackoverflow.com/a/40430328  )
train_data = load_svmlight_file(train_loc)
X_train = train_data[0].toarray()
y_train = train_data[1]

In [9]:
X_train.shape, y_train.shape # ((316281, 85), (316281,))

((316281, 85), (316281,))

In [10]:
# I want to re-use the parameters ..
bundle = joblib.load(f'{datadir}/bundle_with_metrics.joblib')

In [12]:
bundle['input_params']

{'max_depth': 3,
 'learning_rate': 0.1,
 'objective': 'multi:softprob',
 'num_class': 54,
 'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'max_delta_step': 0,
 'min_child_weight': 1,
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': 42,
 'subsample': 1,
 'verbosity': 2}

In [13]:
xgb_model = xgb.XGBClassifier(**bundle['input_params'])


In [18]:
%%time
xgb_model.fit(X_train, y_train)

[18:24:12] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:24:13] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:24:13] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:24:14] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:24:14] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:24:15] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:24:15] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:24:16] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[18:24:16] INFO: src/tree/updater_prune.cc:74: tree prun

[18:24:49] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:24:50] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:24:50] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:24:51] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:24:51] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:24:52] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:24:52] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:24:53] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:24:53] INFO: src/tree/updater_prune.cc:74: tree prun

[18:25:26] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:25:26] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:25:27] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:25:27] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:25:28] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:25:28] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:25:29] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:25:29] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:25:30] INFO: src/tree/updater_prune.cc:74: tree prun

[18:26:03] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[18:26:03] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:26:03] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:26:04] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:26:04] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:26:05] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:26:10] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:26:11] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:26:11] INFO: src/tree/updater_prune.cc:74: tree prun

[18:26:44] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:26:45] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:26:45] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:26:46] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:26:46] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:26:46] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:26:47] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:26:47] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[18:26:48] INFO: src/tree/updater_prune.cc:74: tree prun

...

[19:11:10] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:11:10] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:11:11] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:11:11] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:11:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:11:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:11:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:11:18] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:11:18] INFO: src/tree/updater_prune.cc:74: tree prun

[19:11:51] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:11:52] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:11:52] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:11:53] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:11:53] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:11:54] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:11:54] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:11:55] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:11:55] INFO: src/tree/updater_prune.cc:74: tree prun

[19:12:28] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:12:28] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:12:29] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:12:29] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:12:30] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:12:30] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=3
[19:12:31] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:12:31] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:12:32] INFO: src/tree/updater_prune.cc:74: tree pruni

[19:13:05] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:13:05] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:13:05] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:13:06] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:13:06] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:13:07] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:13:07] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:13:08] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:13:08] INFO: src/tree/updater_prune.cc:74: tree prun

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None, num_class=54,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=42, silent=None, subsample=1,
       verbosity=2)

Mini anecdote, I am observing "top" while this XGBClassifier() is getting fit, using the same parameters as I had used w/ the functional api in [yesterday's notebook](https://github.com/namoopsoo/learn-citibike/blob/2020-revisit/notes/2020-07-03-aws.md)  and I am seeing only `100% CPU` utilization. But I truly recall yesterday seeing `200% CPU` taking advantage of the multiple cores in this environment. I might be mis-remembering, but yesterday's training took 

```
CPU times: user 8min 24s, sys: 1.24 s, total: 8min 26s
Wall time: 4min 18s
```

per the [training](https://github.com/namoopsoo/learn-citibike/blob/2020-revisit/notes/2020-07-03-aws.md#train)  so it is kind of convenient that the magic function `%%time` actually indicates the multi-core time. So I think therefore some parallelization is perhaps available in the functional (raw) api which is not getting taken advantage of w/ the sklearn API? 

This sklearn training is already `30 minutes` and still going . 

ok finished after ..

```
CPU times: user 49min 10s, sys: 1.15 s, total: 49min 11s
Wall time: 49min 6s
```
Wow yea I would like to have more understanding around this parallel behavior.

In [20]:
# Save this classifier just in case... 
workdir = fu.make_work_dir(localdir); print(workdir)
bundle_loc = f'{workdir}/bundle.joblib'
joblib.dump({
    'notebook': '2020-07-04-aws.ipynb',
    'xgb_model': xgb_model,
    'train': 'artifacts/2020-07-03T171842Z/train.libsvm',
    'api': 'sklearn', # 
    'walltime': ''' CPU times: user 49min 10s, sys: 1.15 s, total: 49min 11s
Wall time: 49min 6s''',
    'primary_dataset': '2013-07 - Citi Bike trip data.csv',
    'input_params': bundle['input_params'],
    'proc_bundle': 'artifacts/2020-07-03T171842Z/proc_bundle.joblib',
    'num_round': 'unclear since this is the sklearn api. not sure how to check.'


}, bundle_loc)
print('saved bundle to ', bundle_loc)

/home/ec2-user/SageMaker/learn-citibike/artifacts/2020-07-04T192632Z
saved bundle to  /home/ec2-user/SageMaker/learn-citibike/artifacts/2020-07-04T192632Z/bundle.joblib


In [ ]:
# quick cleanup
# print(fu.get_my_memory())
# del train_data, X_train, y_t
# print(fu.get_my_memory())

# XGBClassifier has "n_estimators=100" .  
# Dont know what that corresponds to in the functional api.


In [22]:
%%time

# Evaluation...
test_loc = f'{datadir}/test.libsvm'
print('evaluate using ', test_loc)
test_data = load_svmlight_file(test_loc)
X_test = test_data[0].toarray()
y_test = test_data[1]
y_prob = xgb_model.predict_proba(X_test)

predictions = np.argmax(y_prob, axis=1)
metrics = {
    'accuracy': accuracy_score(y_test, predictions),
    'balanced_accuracy': balanced_accuracy_score(y_test, predictions),    
    'logloss': fu.big_logloss(y_test, y_prob=y_prob, labels=list(range(54))),
}
metrics

evaluate using  /home/ec2-user/SageMaker/learn-citibike/artifacts/2020-07-03T171842Z/test.libsvm


NameError: name 'y' is not defined

In [ ]:
# And I should note down these numbers for the train data as well for comparison.

In [24]:
y_prob.shape, predictions.shape

((105427, 54), (105427,))

In [25]:
# Wow... kind of creepy these numbers are exactly the same as w/ the functional api.
# So almost like a heuristic proof of some kind that they are equivalent APIs? 
# Maybe shows that `100` rounds are used by default?
# And maybe also showing the parallelization did not deteriorate the results?
metrics = {
    'accuracy': accuracy_score(y_test, predictions),
    'balanced_accuracy': balanced_accuracy_score(y_test, predictions),    
    'logloss': fu.big_logloss(y_test, y_prob=y_prob, labels=list(range(54))),
}
metrics

{'accuracy': 0.11619414381515171,
 'balanced_accuracy': 0.10198056214190813,
 'logloss': 3.477780518331715}

In [27]:
%%time
# check what training accuracy is aas well
print(X_train.shape, y_train.shape)

y_prob = xgb_model.predict_proba(X_train)
predictions = np.argmax(y_prob, axis=1)
train_metrics = {
    'train_accuracy': accuracy_score(y_train, predictions),
    'train_balanced_accuracy': balanced_accuracy_score(y_train, predictions),    
    'train_logloss': fu.big_logloss(y_train, y_prob=y_prob, labels=list(range(54))),
}
c

(316281, 85) (316281,)
CPU times: user 2min 46s, sys: 348 ms, total: 2min 46s
Wall time: 2min 49s


In [28]:
train_metrics

{'train_accuracy': 0.11928949257147915,
 'train_balanced_accuracy': 0.10517465243360108,
 'train_logloss': 3.4663923052147214}

In [ ]:
# Ok training accuracy, and logloss slightly better. So no overfitting here. 
# But underfitting for sure.

In [29]:
X_train.shape

(316281, 85)

In [30]:
# Save this classifier just in case... 
workdir = fu.make_work_dir(localdir); print(workdir)
bundle_loc = f'{workdir}/bundle_with_metrics.joblib'
joblib.dump({
    'notebook': '2020-07-04-aws.ipynb',
    'xgb_model': xgb_model,
    'train': 'artifacts/2020-07-03T171842Z/train.libsvm',
    'api': 'sklearn', # 
    'walltime': ''' CPU times: user 49min 10s, sys: 1.15 s, total: 49min 11s
Wall time: 49min 6s''',
    'primary_dataset': '2013-07 - Citi Bike trip data.csv',
    'input_params': bundle['input_params'],
    'proc_bundle': 'artifacts/2020-07-03T171842Z/proc_bundle.joblib',
    'num_round': 'unclear since this is the sklearn api. not sure how to check.',
    'test_set': 'artifacts/2020-07-03T171842Z/test.libsvm',
    'validation_metrics': metrics,
    'train_metrics': train_metrics,
}, bundle_loc)
print('saved bundle to ', bundle_loc)

/home/ec2-user/SageMaker/learn-citibike/artifacts/2020-07-04T203809Z
saved bundle to  /home/ec2-user/SageMaker/learn-citibike/artifacts/2020-07-04T203809Z/bundle_with_metrics.joblib


In [32]:
y_prob = xgb_model.predict_proba(X_test)


In [39]:
logloss = fu.big_logloss(y_test, y_prob=y_prob, labels=list(range(54)))
# but what would be the worst possible log loss?
predictions = np.argmax(y_prob, axis=1)


In [40]:
print('actual', y_test[0])
print('prediction', predictions[0])
list(zip(y_prob[0], list(range(54))))
 

actual 40.0
prediction 16


[(0.03266318, 0),
 (0.055259336, 1),
 (0.0063982885, 2),
 (0.0060006455, 3),
 (0.028050173, 4),
 (0.013179544, 5),
 (0.0040552644, 6),
 (0.0027082511, 7),
 (0.0068830275, 8),
 (0.045714676, 9),
 (0.042376906, 10),
 (0.009649674, 11),
 (0.002363756, 12),
 (0.02285816, 13),
 (0.008648784, 14),
 (0.027313996, 15),
 (0.063226275, 16),
 (0.0125913145, 17),
 (0.019257562, 18),
 (0.0021036789, 19),
 (0.020691957, 20),
 (0.024331894, 21),
 (0.012355921, 22),
 (0.016046353, 23),
 (0.01745567, 24),
 (0.023194894, 25),
 (0.0043068277, 26),
 (0.012384637, 27),
 (0.009160148, 28),
 (0.035972316, 29),
 (0.02696312, 30),
 (0.013803705, 31),
 (0.01833542, 32),
 (0.01165816, 33),
 (0.0066687753, 34),
 (0.026502948, 35),
 (0.0066653066, 36),
 (0.006679889, 37),
 (0.025716392, 38),
 (0.0044111637, 39),
 (0.008954995, 40),
 (0.01161991, 41),
 (0.022934834, 42),
 (0.04198651, 43),
 (0.0018490063, 44),
 (0.012043682, 45),
 (0.037741266, 46),
 (0.0016105745, 47),
 (0.0514548, 48),
 (0.028019251, 49),
 (0.011

In [46]:
# just double checking predictions add to 1.0 roughly 
Counter(y_prob.sum(axis=1))

Counter({1.0: 53477,
         1.0000001: 17938,
         0.99999994: 27680,
         0.9999999: 5849,
         0.9999998: 314,
         1.0000002: 169})

In [48]:
from sklearn.preprocessing import LabelBinarizer

In [60]:
# ok so best and worst possible logloss...
lb = LabelBinarizer()
lb.fit(range(54))
best_pred_prob = lb.transform(y_test)

best_logloss = fu.big_logloss(y_test, y_prob=best_pred_prob, labels=list(range(54)))
print('best_logloss', best_logloss)

# Worst...
worst_pred_prob_1 = lb.transform([((a + 1) % 54) for a in y_test])
worst_logloss_1 = fu.big_logloss(y_test, y_prob=worst_pred_prob_1, labels=list(range(54)))
print('worst logloss (off by 1 class) ', worst_logloss_1)

worst_pred_prob_2 = lb.transform([((a + 2) % 54) for a in y_test])
worst_logloss_2 = fu.big_logloss(y_test, y_prob=worst_pred_prob_2, labels=list(range(54)))
print('worst logloss (off by 2 classes) ', worst_logloss_2)

worst_pred_prob_3 = lb.transform([((a + 3) % 54) for a in y_test])
worst_logloss_3 = fu.big_logloss(y_test, y_prob=worst_pred_prob_3, labels=list(range(54)))
print('worst logloss (off by 3 classes) ', worst_logloss_3)

best_logloss 5.3858291858115845e-14
worst logloss (off by 1 class)  34.538776394910755
worst logloss (off by 2 classes)  34.53877639491076
worst logloss (off by 3 classes)  34.538776394910755


In [ ]:
# so yea as expected doesnt matter ordinally where the wrong answer is, 
# logloss maxes out maybe related to the dimensionality of the data

# And similarly to an original "top k" approach I had in the first iteration of this project,
# I want to look at how many top k classes do I need to look at to get the correct answer.
# Is it random? What does the accuracy look like if the answers are basically random?

In [71]:
# uniform..
u = (1/54)
uniform_prob = np.array([[u for i in range(54)] for a in y_test])
uniform_logloss = fu.big_logloss(y_test, y_prob=uniform_prob, labels=list(range(54)))
print('uniform_logloss ', uniform_logloss)

# random probabilities... 
random_prob = np.array([(lambda x: x/x.sum())(np.random.random_sample(size=54)) 
                        for _ in y_test])
random_logloss = fu.big_logloss(y_test, y_prob=random_prob, labels=list(range(54)))
print('random_logloss', random_logloss)

random_prob = np.array([(lambda x: x/x.sum())(np.random.random_sample(size=54)) 
                        for _ in y_test])
random_logloss = fu.big_logloss(y_test, y_prob=random_prob, labels=list(range(54)))
print('random_logloss', random_logloss)

random_prob = np.array([(lambda x: x/x.sum())(np.random.random_sample(size=54)) 
                        for _ in y_test])
random_logloss = fu.big_logloss(y_test, y_prob=random_prob, labels=list(range(54)))
print('random_logloss', random_logloss)

uniform_logloss  3.9889840465642754
random_logloss 4.289770557213125
random_logloss 4.292013338213038
random_logloss 4.293046502038444


In [66]:
log_loss(y_test, uniform_prob, labels=list(range(54)))

3.988984046564276

In [69]:
x = np.random.random_sample(size=10)
xnormalized = x/x.sum()
xnormalized.sum(), xnormalized

(1.0000000000000002,
 array([0.09395866, 0.13868692, 0.01335335, 0.06085171, 0.13990607,
        0.13225423, 0.14334201, 0.09554603, 0.13666608, 0.04543494]))